## Импорты

In [55]:
import pandas as pd
from matplotlib import pyplot as plt
from pyvis.network import Network
import networkx as nx

CatmaidInstance

In [56]:
import pymaid

catmaid_url = 'https://l1em.catmaid.virtualflybrain.org'
http_user = None
http_password = None
project_id = 1

rm = pymaid.CatmaidInstance(catmaid_url, http_user, http_password, project_id)

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


## Класс для получения и сохранения структуры одного нейрона

In [ ]:
class simplified_structure:
    def __init__(self, neuron:pymaid.CatmaidNeuronList):
        self.neuron:pymaid.CatmaidNeuronList = neuron
        self.nodes:nx.MultiDiGraph = None
        # названия в формате id_обекта, если знак id положителен, то это нода скелета, если отрицателен то это коннектор
        self.build_structure()
        self.add_connectors_to_graph()
        self.simplify_directed_graph()

    def build_structure(self):
        nodes = self.neuron.nodes
        graph = nx.MultiDiGraph()
        for idt, parent, ntype in zip(nodes['node_id'], nodes['parent_id'], nodes['type']):
            # сома это root
            graph.add_node(idt, type = ntype) 
            graph.add_node(parent, type = ntype)
            graph.add_edge(idt, parent, nodes_inside = [])
        self.nodes = graph

    def add_connectors_to_graph(self):
        connectors = pymaid.get_connectors(self.neuron)
        for idt, ctype in zip(connectors['connector_id'], connectors['type']):
            cID = -idt
            q = self.neuron.connectors[self.neuron.connectors['connector_id'] == idt]
            self.nodes.add_node(cID, type = ctype)
            for node_id in q['node_id']:
                self.nodes.add_edge(cID, node_id)


    def simplify_directed_graph(self):
        def keep_nodes(graph, vid):
            return graph.nodes(True)[vid]['type'] == 'root' or vid < 0
        G = self.nodes
        original = len(self.nodes)
        while True:
            # Находим все вершины с in-degree=1 и out-degree=1
            nodes_to_remove = [
                node for node in G.nodes() 
                if G.in_degree(node) == 1 and G.out_degree(node) == 1 and not keep_nodes(G, node)
            ]
            
            if not nodes_to_remove:
                break # Если таких вершин нет, завершаем

            for node in nodes_to_remove:
                # Если узел уже был удален на предыдущей итерации этого же цикла, пропускаем
                if node not in G: 
                    continue

                # Получаем единственного предшественника и преемника
                # NetworkX гарантирует, что list(predecessors/successors) вернет один элемент,
                # если степень равна 1.
                u = list(G.predecessors(node))
                v = list(G.successors(node))

                if len(u) != 1 or len(v) != 1:
                    raise Exception('Этот эксепшен не должен никогда вызватся, но он вызвался и значит что то пошло не так')
                u = u[0]
                v = v[0]

                nodes_inside = sum((edge[-1]['nodes_inside'] for edge in G.edges(node, data = True)), [])
                    
                if u != v:
                    G.add_edge(u, v, nodes_inside = [node] + nodes_inside)
                
                G.remove_node(node)

        after = len(self.nodes)
        print('removed', original - after, 'nodes.', f'Efficiency: {round(100*(1 - after/original), 1)}%')
    
    def save_as_pyvis_html(self, path):
        net = Network(notebook = False, directed = True)
        net.from_nx(self.nodes)
        for node in net.nodes:
            node['label'] = node['type']
            if node['type'] == 'root':
                node['size'] = 30
            if node['id'] < 0:
                node['color'] = 'orange'
                node['size'] = 5

        net.show_buttons(filter_=['physics'])
        net.save_graph(path)

## Тестируем simplified_structure

In [58]:
A = pymaid.get_neuron(29)
B = pymaid.get_neuron(9469519)

In [59]:
S = simplified_structure(A)
S.save_as_pyvis_html(f'visualizaions/test/graph_of_{A.id}.html')

removed 2443 nodes. Efficiency: 75.7%


## Класс для объединения структур нескольких нейронов

In [60]:
a = simplified_structure(A)
b = simplified_structure(B)

INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


removed 2443 nodes. Efficiency: 75.7%


INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


removed 3615 nodes. Efficiency: 76.3%


In [ ]:
#a.nodes * b.nodes
#nx.intersection(a.nodes, b.nodes).nodes
len(nx.compose(a.nodes, b.nodes))

1911


1909

## Класс для визуализации